<a href="https://colab.research.google.com/github/Alimustoofaa/DatasetContainerNumber/blob/master/Container_Number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Detectron2

In [ ]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import matplotlib.pyplot as plt

from detectron2.modeling import build_model
from detectron2.config import get_cfg
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # Set threshold for this model
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/HALOTEC/CONTAINER_NUMBER_V2/Faster_RNN/output/model_final.pth' # Set model .pth
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
predictor = DefaultPredictor(cfg)

In [ ]:
im = cv2.imread('/content/1907172159070010610.jpg')
outputs = predictor(im)

try:
  v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  plt.figure(figsize = (14, 10))
  plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
  plt.show()
except:
  print("no boxes detected")

In [ ]:
import uuid 

boxes = {}
score_array = []
img_crop_array = []
for i, coordinates in enumerate(outputs["instances"].to("cpu").pred_boxes, start=0):
  score_array.append(outputs["instances"].scores[i].item())
  coordinates_array = []
  for k in coordinates:
    coordinates_array.append(int(k))
  boxes[uuid.uuid4().hex[:].upper()] = coordinates_array

for k,v in boxes.items():
  crop_img = im[v[1]-13:v[3]+13, v[0]-13:v[2]+13, :]
  img_crop_array.append(crop_img)

score_hight = max(range(len(score_array)), key=score_array.__getitem__)
img_detection = cv2.cvtColor(img_crop_array[score_hight], cv2.COLOR_BGR2RGB)
print('Score : ', score_array[score_hight])

plt.imshow(img_detection)
plt.show()
plt.imsave('test.jpg', img_detection)
